## Robinhood Order History
Parse robinhood orders API response, flatten to get list of all orders and enrich by adding stock symbol and name

In [1]:
import pandas as pd
import urllib.request, json

In [2]:
orders = pd.read_json('orders_sample.json', orient='columns')
orders = orders['results']

In [3]:
orders

0    {'updated_at': '2018-12-21T19:31:18.339622Z', ...
1    {'updated_at': '2018-12-21T19:30:23.297759Z', ...
2    {'updated_at': '2018-12-21T19:29:06.588381Z', ...
3    {'updated_at': '2017-08-10T14:15:46.907458Z', ...
4    {'updated_at': '2017-10-19T14:12:55.200328Z', ...
5    {'updated_at': '2017-08-10T14:19:23.318570Z', ...
Name: results, dtype: object

In [4]:
orders_normalize = pd.io.json.json_normalize(orders, record_path=['executions'], 
                                        meta=['instrument', 'side'], 
                                        sep='_')
orders_normalize

,id,price,quantity,settlement_date,timestamp,instrument,side
0,12d7sfc5-4f40-4c02-be91-fc5260765af7,149.19990000,10.00000,2018-12-26,2018-12-21T19:31:18.099000Z,https://api.robinhood.com/instruments/1790dd4f...,buy
1,6d3f9f83-c62d-4da1-8321-40804bc4cc3a,75.46000000,15.00000,2018-12-26,2018-12-21T19:30:23.072000Z,https://api.robinhood.com/instruments/e6a6e495...,buy
2,87426fee-234b-4760-b50c-8539a8b58464,106.77720000,10.00000,2018-12-26,2018-12-21T19:29:06.327000Z,https://api.robinhood.com/instruments/7fcf5e7b...,buy
3,e39b50f4-3aea-4235-8459-ff082a5af70d,142.42000000,15.00000,2017-08-15,2017-08-10T14:15:40.820000Z,https://api.robinhood.com/instruments/1790dd4f...,buy
4,65695025-251b-43d9-8aaf-bbd899a4623e,26.31100000,75.00000,2017-10-23,2017-10-19T14:12:55.165000Z,https://api.robinhood.com/instruments/0dd811b3...,sell
5,3f5d00e3-3295-43f9-bbc6-2f053164901f,24.39900000,75.00000,2017-08-15,2017-08-10T14:19:23.285000Z,https://api.robinhood.com/instruments/0dd811b3...,buy


### Get instrument name and symbol by querying instrument API

In [5]:
unique_instruments = pd.DataFrame(orders_normalize['instrument'].unique(), columns=['instrument'])
unique_instruments

,instrument
0,https://api.robinhood.com/instruments/1790dd4f...
1,https://api.robinhood.com/instruments/e6a6e495...
2,https://api.robinhood.com/instruments/7fcf5e7b...
3,https://api.robinhood.com/instruments/0dd811b3...


In [6]:
def get_instrument_name(instrument_url):
    with urllib.request.urlopen(instrument_url) as url:
        data = json.loads(url.read().decode())
        return data['simple_name'], data['symbol']

In [7]:
unique_instruments[['name', 'symbol']] = unique_instruments.apply(lambda x:get_instrument_name(x['instrument']), axis=1, result_type="expand")
unique_instruments

,instrument,name,symbol
0,https://api.robinhood.com/instruments/1790dd4f...,PowerShares QQQ,QQQ
1,https://api.robinhood.com/instruments/e6a6e495...,Vanguard US REIT fund,VNQ
2,https://api.robinhood.com/instruments/7fcf5e7b...,Digital Realty Trust,DLR
3,https://api.robinhood.com/instruments/0dd811b3...,Bank of America,BAC


### All orders along with name & symbol

In [8]:
all_orders = pd.merge(orders_normalize, unique_instruments, how='inner', on='instrument')
all_orders = all_orders[['symbol','name','price','quantity','side','timestamp','settlement_date']]
all_orders['timestamp'] = pd.to_datetime(all_orders['timestamp']).dt.strftime('%Y-%m-%d %H:%M')
all_orders.rename(columns={'timestamp':'purchase_date'}, inplace=True)
all_orders

,symbol,name,price,quantity,side,purchase_date,settlement_date
0,QQQ,PowerShares QQQ,149.19990000,10.00000,buy,2018-12-21 19:31,2018-12-26
1,QQQ,PowerShares QQQ,142.42000000,15.00000,buy,2017-08-10 14:15,2017-08-15
2,VNQ,Vanguard US REIT fund,75.46000000,15.00000,buy,2018-12-21 19:30,2018-12-26
3,DLR,Digital Realty Trust,106.77720000,10.00000,buy,2018-12-21 19:29,2018-12-26
4,BAC,Bank of America,26.31100000,75.00000,sell,2017-10-19 14:12,2017-10-23
5,BAC,Bank of America,24.39900000,75.00000,buy,2017-08-10 14:19,2017-08-15


### Export data as CSV

In [9]:
all_orders.to_csv('orders.csv', encoding='utf-8', index=False)